In [10]:
# PhoBERT NER Simple Training Script with Debug Prints (Label-as-String)

import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import Dataset, DatasetDict
import evaluate
import numpy as np

# Define labels as strings (no mapping to IDs)
label_list = ["O", "B-PRODUCT", "I-PRODUCT", "B-PRICE", "I-PRICE", "B-COMPARE", "I-COMPARE"]

# Load PhoBERT tokenizer and model using AutoTokenizer with fast=True
model_checkpoint = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

# Read and parse train.txt
print("Reading train.txt...")
with open("./data/train.txt", "r", encoding="utf-8") as f:
    lines = f.read().strip().split("\n")

tokens = []
labels = []
sentence_tokens = []
sentence_labels = []

for line in lines:
    if line == "":
        if sentence_tokens:
            tokens.append(sentence_tokens)
            labels.append(sentence_labels)
            sentence_tokens = []
            sentence_labels = []
    else:
        splits = line.strip().split()
        if len(splits) == 2:
            token, label = splits
            sentence_tokens.append(token)
            sentence_labels.append(label)

# Add last sentence if needed
if sentence_tokens:
    tokens.append(sentence_tokens)
    labels.append(sentence_labels)

train_examples = {"tokens": tokens, "ner_tags": labels}
print(train_examples)

# Create DatasetDict
dataset = DatasetDict({
    "train": Dataset.from_dict(train_examples),
    "test": Dataset.from_dict(train_examples),
})

# Tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append("O")
            else:
                label_ids.append(label[word_idx])
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

print("Tokenizing dataset...")
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Map string labels to IDs for model
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}

def convert_labels_to_ids(examples):
    examples["labels"] = [[label_to_id[label] for label in seq] for seq in examples["labels"]]
    return examples

tokenized_datasets = tokenized_datasets.map(convert_labels_to_ids, batched=True)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list)).to(device)
data_collator = DataCollatorForTokenClassification(tokenizer)

# Evaluation metric
metric = evaluate.load("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id_to_label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    print("Evaluation results:", results)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Training arguments
training_args = TrainingArguments(
    output_dir="./phobert-ner",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=10,
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train the model
print("Starting training...")
trainer.train()
print("Training completed!")


Reading train.txt...
{'tokens': [['Tôi', 'muốn', 'mua', 'điện', 'thoại', 'dưới', '200', 'đô'], ['Tìm', 'máy', 'lạnh', 'giá', 'trên', '10', 'triệu'], ['Có', 'tủ', 'lạnh', 'nào', 'dưới', '5', 'triệu', 'không'], ['Tôi', 'cần', 'một', 'laptop', 'trên', '15', 'triệu'], ['Cho', 'hỏi', 'máy', 'giặt', 'dưới', '300', 'đô', 'có', 'không'], ['Tìm', 'máy', 'tính', 'bảng', 'giá', 'dưới', '7', 'triệu'], ['Mình', 'đang', 'tìm', 'tivi', 'trên', '12', 'triệu'], ['Có', 'máy', 'ảnh', 'nào', 'giá', 'dưới', '10', 'triệu', 'không'], ['Mua', 'loa', 'bluetooth', 'dưới', '1', 'triệu'], ['Mình', 'muốn', 'mua', 'điện', 'thoại', 'giá', 'cao', 'hơn', '10', 'triệu'], ['Tôi', 'muốn', 'mua', 'điện', 'thoại', 'dưới', '200', 'đô'], ['Tìm', 'máy', 'lạnh', 'giá', 'trên', '10', 'triệu'], ['Có', 'tủ', 'lạnh', 'nào', 'dưới', '5', 'triệu', 'không'], ['Tôi', 'cần', 'một', 'laptop', 'trên', '15', 'triệu'], ['Cho', 'hỏi', 'máy', 'giặt', 'dưới', '300', 'đô', 'có', 'không'], ['Tìm', 'máy', 'tính', 'bảng', 'giá', 'dưới', '7', 'tri

Map:   0%|          | 0/80 [00:00<?, ? examples/s]


ValueError: word_ids() is not available when using non-fast tokenizers (e.g. instance of a `XxxTokenizerFast` class).

In [52]:
import pandas as pd
from datasets import Dataset

# Đọc tệp CSV chứa dữ liệu
data = pd.read_csv('./data/train_data.csv')

# Chia dữ liệu thành các câu và nhãn
sentences = []
labels = []
sentence = []
label = []

# Duyệt qua từng dòng trong CSV để tạo ra danh sách câu và nhãn
for idx, row in data.iterrows():
    token = row['Token']
    tag = row['Label']
    
    # Thêm token vào câu và nhãn
    sentence.append(token)
    label.append(tag)
    
    # Khi gặp dòng cuối cùng của mỗi câu (có thể tùy chỉnh ở đây)
    if token == 'triệu' or token == 'đô' or token == 'nghìn'  :  # Chỉ là ví dụ, điều chỉnh phù hợp với cấu trúc của bạn
        sentences.append(sentence)
        labels.append(label)
        sentence = []
        label = []

# Chuyển đổi thành định dạng Dataset
dataset = Dataset.from_dict({
    'tokens': sentences,
    'ner_tags': labels
})

# In ra mẫu
print(dataset[:3])


{'tokens': [['Tôi', 'muốn', 'mua', 'điện thoại', 'dưới', '200', 'đô'], ['Tìm', 'máy lạnh', 'giá', 'trên', '10', 'triệu'], ['Có', 'tủ lạnh', 'nào', 'dưới', '5', 'triệu']], 'ner_tags': [['O', 'O', 'O', 'B-PRODUCT', 'B-COMPARE', 'B-PRICE', 'I-PRICE'], ['O', 'B-PRODUCT', 'O', 'O-COMPARE', 'B-PRICE', 'I-PRICE'], ['O', 'B-PRODUCT', 'O', 'B-COMPARE', 'B-PRICE', 'I-PRICE']]}


In [27]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
phobert = AutoModelForTokenClassification.from_pretrained("vinai/phobert-base-v2").to(device)
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

# Your label mapping (directly using label values, not ID)
label_list = ["O", "B-PRODUCT", "I-PRODUCT", "B-PRICE", "I-PRICE", "B-COMPARE", "I-COMPARE"]

# Word-segmented input sentence
sentence = "Tôi muốn mua điện thoại dưới 200 đô"

# Tokenize
tokens = tokenizer(sentence, return_tensors="pt", truncation=True).to(device)
with torch.no_grad():
    outputs = phobert(**tokens)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2)

# Convert token IDs to corresponding labels
token_ids = tokens["input_ids"][0]
predicted_labels = [label_list[pred.item()] for pred in predictions[0]]

# Decode tokens
decoded_tokens = tokenizer.convert_ids_to_tokens(token_ids)

# Print the results
for token, label in zip(decoded_tokens, predicted_labels):
    print(f"{token:<20} {label}")


Using device: cuda


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


<s>                  O
Tôi                  B-PRODUCT
muốn                 B-PRODUCT
mua                  B-PRODUCT
điện                 O
thoại                O
dưới                 B-PRODUCT
200                  B-PRODUCT
đô                   O
</s>                 B-PRODUCT


In [57]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification
from torch.optim import AdamW
from datasets import Dataset, DatasetDict
import evaluate
from tqdm import tqdm

# --- Cấu hình ---
label_list = ["O", "B-PRODUCT", "I-PRODUCT", "B-PRICE", "I-PRICE", "B-COMPARE", "I-COMPARE"]
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for label, i in label_to_id.items()}
model_checkpoint = "vinai/phobert-base"

# --- Load tokenizer và model ---
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

# --- Tạo dữ liệu ---
train_examples = {
    "tokens": [
        ["Tôi", "muốn", "mua", "điện", "thoại", "dưới", "200", "đô"],
        ["Tìm", "máy", "lạnh", "giá", "trên", "10", "triệu"],
        ["Có", "tủ", "lạnh", "nào", "dưới", "5", "triệu", "không"]
    ],
    "ner_tags": [
        ["O", "O", "O", "B-PRODUCT", "I-PRODUCT", "B-COMPARE", "B-PRICE", "I-PRICE"],
        ["O", "B-PRODUCT", "I-PRODUCT", "O", "B-COMPARE", "B-PRICE", "I-PRICE"],
        ["O", "B-PRODUCT", "I-PRODUCT", "O", "B-COMPARE", "B-PRICE", "I-PRICE", "O"]
    ]
}

raw_datasets = DatasetDict({
    "train": Dataset.from_dict(train_examples),
    "test": Dataset.from_dict(train_examples),
})

# --- Tiền xử lý và căn chỉnh nhãn thủ công ---
def tokenize_and_align_labels(examples):
    tokenized_inputs = {"input_ids": [], "attention_mask": [], "labels": []}

    for tokens, labels in zip(examples["tokens"], examples["ner_tags"]):
        # Sử dụng tokenizer để chuyển các token thành input_ids và attention_mask
        encoding = tokenizer(tokens, truncation=True, padding=True, max_length=128, is_split_into_words=True)

        input_ids = encoding["input_ids"]
        attention_mask = encoding["attention_mask"]

        # Cập nhật nhãn (label) cho các sub-token
        label_ids = []
        for token, label in zip(tokens, labels):
            word_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(token))
            label_ids.append(label_to_id[label])
            label_ids.extend([-100] * (len(word_ids) - 1))  # Đánh dấu các sub-token phụ thuộc

        # Thêm nhãn cho padding token
        label_ids = label_ids[:128]  # Cắt cho phù hợp với max_length (tránh vượt quá giới hạn)

        tokenized_inputs["input_ids"].append(input_ids)
        tokenized_inputs["attention_mask"].append(attention_mask)
        tokenized_inputs["labels"].append(label_ids)

    return tokenized_inputs

print("Tokenizing...")
tokenized_datasets = raw_datasets.map(tokenize_and_align_labels, batched=True)

# --- DataCollator ---
data_collator = DataCollatorForTokenClassification(tokenizer)

# --- DataLoader ---
train_loader = DataLoader(tokenized_datasets["train"], batch_size=8, shuffle=True, collate_fn=data_collator)
test_loader = DataLoader(tokenized_datasets["test"], batch_size=8, collate_fn=data_collator)

# --- Huấn luyện ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
metric = evaluate.load("seqeval")

# Hàm đánh giá mô hình
def evaluate_model():
    model.eval()
    all_preds, all_labels = [], []
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        labels = batch["labels"]
        for pred, label in zip(predictions, labels):
            true_pred = [id_to_label[p.item()] for (p, l) in zip(pred, label) if l != -100]
            true_label = [id_to_label[l.item()] for (p, l) in zip(pred, label) if l != -100]
            all_preds.append(true_pred)
            all_labels.append(true_label)
    result = metric.compute(predictions=all_preds, references=all_labels)
    print("Evaluation:", result)

print("Training...")
model.train()
for epoch in range(5):
    print(f"Epoch {epoch+1}")
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    evaluate_model()

print("Training done.")
model.save_pretrained("./phobert-ner")
tokenizer.save_pretrained("./phobert-ner")


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing...


Map: 100%|██████████| 3/3 [00:00<00:00, 501.25 examples/s]


Training...
Epoch 1


  0%|          | 0/1 [00:00<?, ?it/s]


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`tokens` in this case) have excessive nesting (inputs type `list` where type `int` is expected).